In [351]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Data

In [352]:
datapath = "dataset/"

df = pd.read_csv(datapath + 'data.csv')

In [353]:
df.head()

,hPossesion,aPossesion,hshotsOnTarget,ashotsOnTarget,hshots,ashots,hfouls,afouls,hsaves,asaves,label
0,24,76,2,4,14,17,11,13,2,1,2
1,80,20,3,1,5,2,1,3,0,1,1
2,57,43,4,2,12,11,10,12,1,2,1
3,47,53,5,3,13,11,7,10,1,3,0
4,72,28,5,4,21,6,8,13,4,5,0


In [354]:
df.shape

(1272, 11)

# Nueral Net From Scratch

### Selected Data to Train

In [355]:
sample_df = df.sample(n=100, random_state=42)
sample_df.head()

,hPossesion,aPossesion,hshotsOnTarget,ashotsOnTarget,hshots,ashots,hfouls,afouls,hsaves,asaves,label
208,50,50,7,0,11,2,7,7,0,3,1
966,45,55,1,3,9,10,20,18,3,1,0
714,37,63,2,1,8,10,13,10,1,1,1
1192,64,36,15,3,20,6,15,13,1,7,0
584,28,72,0,5,3,22,12,4,3,0,2


In [356]:
train, test = np.split(sample_df.sample(frac=1, random_state=42), [int(0.8 * len(sample_df))])

train.shape, test.shape

c:\Users\Public\Anaconda\envs\dsde-cp\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


((80, 11), (20, 11))

In [357]:
X = train.drop(columns=['label'])
y = train['label']

X_test = test.drop(columns=['label'])
y_test = test['label']

X.shape, y.shape

((80, 10), (80,))

## init weights

In [358]:
W = np.random.randn(10, 3)
W

array([[ 0.16694146,  2.031977  , -0.84396551],
       [-0.55969476,  0.85682239,  0.05789393],
       [ 0.22604722, -0.7559193 ,  0.76939618],
       [ 0.22737514, -0.30728726, -0.78125198],
       [ 1.5850684 ,  0.21913624, -0.5119989 ],
       [-1.18858641, -0.75220944,  1.22944987],
       [-0.9541277 , -0.1361906 , -0.61839602],
       [ 0.13324106,  2.36096963, -0.81831754],
       [-0.3216835 , -0.27641603,  2.50016126],
       [-0.32606433,  0.19240507,  0.63334637]])

In [359]:
W0 = np.random.randn(1, 3)
W0

array([[-0.82486426, -1.52813274, -1.12871573]])

## Forward Pass

In [360]:
# Forward pass
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / exp_z.sum(axis=1, keepdims=True)

## Original Evaluation

In [361]:
logits = (X @ W + W0).to_numpy()  
probs = softmax(logits)         
preds = np.argmax(probs, axis=1) 

In [362]:
preds

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [363]:
original_accuaracy = accuracy_score(y, preds)

print(f"Original Accuracy: {original_accuaracy:.2f}")

Original Accuracy: 0.45


In [364]:
eval_logits = (X_test @ W + W0).to_numpy()  
eval_probs = softmax(eval_logits)
eval_preds = np.argmax(eval_probs, axis=1)
origianl_eval_accuracy = accuracy_score(y_test, eval_preds)
print(f"Original Test Accuracy: {origianl_eval_accuracy:.2f}")

Original Test Accuracy: 0.60


# Training

### Loss Fuction

In [365]:
def cross_entropy_loss(probs, y_true):
    n = y_true.shape[0]
    log_probs = -np.log(probs[range(n), y_true])
    return np.sum(log_probs) / n

### One-hot encoder
class 0 -> [1, 0, 0]

class 1 -> [0, 1, 0]

class 2 -> [0, 0, 1]


In [366]:
def one_hot(y, num_classes):
    return np.eye(num_classes)[y]

In [367]:
learning_rate = 0.1
epochs = 1000
num_classes = 3
n_samples, n_features = X.shape

for epoch in range(epochs + 1):
    # Forward
    logits = np.dot(X, W) + W0
    probs = softmax(logits)
    loss = cross_entropy_loss(probs, y)

    # Back Propagate
    y_onehot = one_hot(y, num_classes)
    dlogits = (probs - y_onehot) / n_samples
    dW = np.dot(X.T, dlogits)
    dw0 = np.sum(dlogits, axis=0, keepdims=True)

    # Update Weights
    W -= learning_rate * dW
    W0 -= learning_rate * dw0

    # Log
    if epoch % 100 == 0:
        predictions = np.argmax(probs, axis=1)
        acc = np.mean(predictions == y) 
        eval_logits = np.dot(X_test, W) + dw0
        eval_probs = softmax(eval_logits)
        eval_preds = np.argmax(eval_probs, axis=1)
        eval_accuracy = accuracy_score(y_test, eval_preds)
        print(f"Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {acc:.2f}, Eval Accuracy: {eval_accuracy:.2f}")

Epoch 0, Loss: 97.0677, Accuracy: 0.45, Eval Accuracy: 0.25
Epoch 100, Loss: 84.3332, Accuracy: 0.50, Eval Accuracy: 0.10
Epoch 200, Loss: 98.7305, Accuracy: 0.68, Eval Accuracy: 0.55
Epoch 300, Loss: 40.2099, Accuracy: 0.29, Eval Accuracy: 0.60
Epoch 400, Loss: 50.0479, Accuracy: 0.50, Eval Accuracy: 0.40
Epoch 500, Loss: 38.4365, Accuracy: 0.64, Eval Accuracy: 0.40
Epoch 600, Loss: 62.9409, Accuracy: 0.60, Eval Accuracy: 0.20
Epoch 700, Loss: 44.3492, Accuracy: 0.33, Eval Accuracy: 0.55
Epoch 800, Loss: 73.6104, Accuracy: 0.62, Eval Accuracy: 0.45
Epoch 900, Loss: 21.8183, Accuracy: 0.66, Eval Accuracy: 0.55
Epoch 1000, Loss: 84.0912, Accuracy: 0.70, Eval Accuracy: 0.45


# Evalution

In [368]:
logits = np.dot(X_test, W) + dw0
probs = softmax(logits)
preds = np.argmax(probs, axis=1)
accuracy = accuracy_score(y_test, preds)
print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 0.45


# Inferrence

In [369]:
# Inter Milan
homeTeamName = "Inter Milan"
homeAveragePossesion = 56.5
homeAverageShots = 14.6
homeAverageShotsOnTarget = 4.9
homeAverageFouls = 10.8
homeAverageSaves = 5

# PSG
awayTeamName = "PSG"
awayAveragePossesion = 66.6
awayAverageShots = 18.8
awayAverageShotsOnTarget = 7.9
awayAverageFouls = 10
awayAverageSaves = 3

result_map = {
    0: "Draw",
    1: f"{homeTeamName} Win",
    2: f"{awayTeamName} Win"
}

X = np.array([[homeAveragePossesion, awayAveragePossesion, homeAverageShotsOnTarget, awayAverageShotsOnTarget, homeAverageShots, awayAverageShots, homeAverageFouls, awayAverageFouls, homeAverageSaves, awayAverageSaves]])
logits = np.dot(X, W) + W0
probs = softmax(logits)
preds = np.argmax(probs, axis=1)
print(f"Predicted class: {preds[0]}")
print(f"Predicted result: {result_map[preds[0]]}")
print(f"Predicted probabilities: {[f'{p:.2f}' for p in probs[0]]}")
print(f'Model Accuracy: {accuracy:.2f}')

 

Predicted class: 2
Predicted result: PSG Win
Predicted probabilities: ['0.00', '0.00', '1.00']
Model Accuracy: 0.45


In [370]:
print(f'Original Accuracy: {original_accuaracy:.2f}')
print(f'Original Test Accuracy: {origianl_eval_accuracy:.2f}')
print(f'Final Test Accuracy: {accuracy:.2f}')
print(f'Final Test Accuracy: {eval_accuracy:.2f}')

Original Accuracy: 0.45
Original Test Accuracy: 0.60
Final Test Accuracy: 0.45
Final Test Accuracy: 0.45


In [371]:
W

array([[-0.17517234,  1.84890635, -0.31878107],
       [-0.44505054, -0.69061424,  1.49068634],
       [-3.88451743, 13.67214381, -9.54810228],
       [-5.5506256 , -1.93700301,  6.62646452],
       [ 1.26666143,  1.609318  , -1.58377369],
       [-1.77646157, -7.61381276,  8.67892835],
       [-2.01771861,  2.40524585, -2.09624156],
       [ 0.94594438,  0.42473067,  0.30521809],
       [ 5.30211636,  5.45931995, -8.85937458],
       [ 2.04690986, -4.92844405,  3.3812213 ]])